In [1]:
import os
import sys
import numpy as np

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from analysis.RP_privacy_analysis_advanced import rp_find_rho_for_delta, smooth_leverage_upper_bound, f_smooth
from RP_mechanisms.optim_RP_DP import compute_largest_l2

In [2]:
from Dataset.flight_dataset_for_LS import load_flight_dataset_matrices_for_LS, save_flight_dataset_matrices_for_LS

file_X_path = os.path.join(project_dir, "Dataset", "flight-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "flight-LR-y.txt")

X = None
y = None
try:
    X, y = load_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    save_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_flight_dataset_matrices_for_LS()
    
B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

largest_row_norm = compute_largest_l2(A)

In [60]:
r = 10
epsilon = 10
delta = 1e-2

epsilon_1 = epsilon/1.02 
epsilon_2 = epsilon - epsilon_1
delta_1 = delta/1.02
delta_2 = (delta - delta_1) / np.exp(epsilon_1)

rho_1 = rp_find_rho_for_delta(delta_1, epsilon_1, r)
rho_2 = rp_find_rho_for_delta(delta_2, epsilon_2, r)

a = rho_1 / (rho_1 - 1)
t = 1/np.sqrt(rho_2)

print(f"a is {a} with rho_1: {rho_1} and epsilon_1: {epsilon_1} and delta_1: {delta_1}")
print(f"t is {t} with rho_2: {rho_2} and epsilon_2: {epsilon_2} and delta_2: {delta_2}")

ValueError: f(a) and f(b) must have different signs

In [61]:
r = 300
epsilon = 2
delta = 1e-6

rho = rp_find_rho_for_delta(delta, epsilon, r)
a = rho / (rho - 1)


print(f"a is {rho / (rho - 1)} with rho: {rho} and epsilon: {epsilon} and delta: {delta}")

a is 31.21592681344737 with rho: 1.0330951291408 and epsilon: 2 and delta: 1e-06


In [62]:
b = 2
s = smooth_leverage_upper_bound(A, largest_row_norm, b=b)
t = 0.95

f_smooth(a=a, b=b, t=t, ell=largest_row_norm, p=s)

np.float64(0.3307580404962598)

In [48]:
from RP_mechanisms.optim_RP_DP import OptimalRP_mech
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : r,
        'CI' : 0.95,
        'bootstrap_samples': 100,
        'l2' : largest_row_norm
    }
    
    return kwargs

kwargs = generate_default_configuration()

mech = OptimalRP_mech(kwargs)

sigma = mech.find_minimal_sigma(epsilon, delta)
print(sigma)

0.40902269718382395


/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in scalar multiply
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
